In [64]:
import numpy as np
class GQA:
    def __init__(self, d_model, num_heads, num_groups, seed=0):
        assert num_heads % num_groups == 0
        self.d_model = d_model
        self.h = num_heads
        self.g = num_groups
        self.d_k = d_model // num_heads
        # Projections
        self.W_Q = np.random.randn(d_model, self.h * self.d_k) / np.sqrt(d_model)
        self.W_K = np.random.randn(d_model, self.g * self.d_k) / np.sqrt(d_model)
        self.W_V = np.random.randn(d_model, self.g * self.d_k) / np.sqrt(d_model)

        # output projection
        self.W_O = np.random.randn(self.h * self.d_k, d_model) / np.sqrt(self.h * self.d_k)

        # mapping head
        self.group_size = self.h // self.g
        self.head2group = (np.arange(self.h) // self.group_size).astype(int)
    
    def softmax(self, x, axis= -1):
        # ổn định số học: trừ đi max để tránh overflow
        x_max = np.max(x, axis=axis, keepdims=True)
        e_x = np.exp(x - x_max)
        return e_x / np.sum(e_x, axis=axis, keepdims=True)
    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        batch, seq_len, _ = x.shape
        # Linear projections
        Q = x @ self.W_Q
        K = x @ self.W_K
        V = x @ self.W_V

        # reshape thành (batch, h, seq_len, d_k) và 
        # (batch, g, seq_len, d_k)
        Q = Q.reshape(batch, seq_len, self.h, self.d_k).transpose(0, 2, 1, 3)
        K = K.reshape(batch, seq_len, self.g, self.d_k).transpose(0, 2, 1, 3)
        V = V.reshape(batch, seq_len, self.g, self.d_k).transpose(0, 2, 1, 3)

        # Attention cho từng head -> ánh xạ sang group
        outputs = []
        for i in range(self.h):
            g = self.head2group[i]

            Qi = Q[:, i, :, :] #(batch, seq_len, d_k)
            Ki = K[:, g, :, :] #(batch, seq_len, d_k)
            Vi = V[:, g, :, :] #(batch, seq_len, d_k)
            
            #attention score: (batch, seq_len, seq_len)
            scores = Qi @ Ki.transpose(0, 2, 1) / np.sqrt(self.d_k)
            weights = self.softmax(scores, axis=-1)

            # output head i: (batch, seq_len, d_k)
            Oi = weights @ Vi
            outputs.append(Oi)
        
        # Nối tất cả các heads lại
        O = np.concatenate(outputs, axis=-1)

        # Output projection
        O = O @ self.W_O
        return O
            

In [65]:
d_model = 8
num_heads = 4
d_k = d_model // num_heads
h = num_heads
g = 2
W_Q = np.random.randn(d_model, d_model)
print(W_Q)
print(f"--------------------------")
W_K = np.random.randn(d_model, g * d_k)
print(W_K)
print(f"------------------------------")
W_V = np.random.randn(d_model, g * d_k)
print(W_V)

[[ 0.23940519  1.66111856 -0.20356129  1.27058707 -2.94029029  0.8777846
  -0.12489429  1.92314266]
 [-0.4456467   0.48224981  0.47329157 -0.39512664  2.10103225 -0.28560011
  -0.82652432 -0.87472136]
 [ 0.24301883 -0.07056784  0.62040455  0.92186213  1.77013652  1.36108775
  -0.12992348 -0.51241304]
 [-0.34504311  0.01465683  0.31928923 -0.74526591  1.02999523  0.37366542
   0.22558793 -0.72086229]
 [ 0.75736167  0.16427002  0.29324152  0.62370667  0.93508755  0.47297128
   0.18201437 -0.52452746]
 [-0.08815476  1.02038747  0.35872427 -0.26066375  0.74813042  1.63799562
   1.81351774 -0.77223045]
 [ 0.02916833 -0.7436708   0.3835143  -0.40728074  0.08124833 -0.9151844
   2.81192096  0.85253405]
 [-0.17408235 -1.38722378  0.11477672 -1.52247573  0.76825095 -1.29077737
  -0.25602256 -1.54170593]]
--------------------------
[[-0.46401799  0.62588077 -0.01227908 -1.2687157 ]
 [-1.15596893  1.32099733 -2.94466544  2.30214876]
 [-0.5502698  -0.7877159  -1.64055857 -0.14470979]
 [ 1.08922568

In [66]:
group_size = h//g
print(f"Chia {h} heads thành {group_size} nhóm")
head2group = [0] * num_heads
for i in range(num_heads):
    head2group[i] = i // group_size
print(f"Group tương ứng của của các head:\n{head2group}")
for i in range(num_heads):
    print(f"group của head {i} là: {head2group[i]}")

Chia 4 heads thành 2 nhóm
Group tương ứng của của các head:
[0, 0, 1, 1]
group của head 0 là: 0
group của head 1 là: 0
group của head 2 là: 1
group của head 3 là: 1


In [67]:
# Giả sử đầu vào x có 2 mẫu, mỗi mẫu có 3 từ 
# và d_model = d_model = 8
batch = 2
seq_len = 3
x = np.random.randn(batch, seq_len, d_model)
print(f"Đầu vào x:\n{x}")

Đầu vào x:
[[[-0.2503341  -0.55224965 -0.82941883 -0.32478957 -1.03477907
   -0.10103258  0.67859904 -0.69633028]
  [ 1.93484307 -1.29960365 -0.82017787 -1.43283317 -0.48123886
    1.03548408  0.66645496 -1.00530665]
  [-0.2583375   0.04666427  0.03574593  0.16737548 -2.04831901
   -0.60374309  2.70460309 -0.32030509]]

 [[ 0.91575715  3.14560031 -0.59519807  1.14812669 -0.43936931
   -1.31838479  1.40468864 -0.05228306]
  [ 0.34348862 -0.48894452 -0.76577516  0.35691139 -1.243329
   -0.40908092  0.83570525  0.79832422]
  [ 0.07816402 -1.10923177  1.44630279  0.2716136  -0.3219902
   -0.62180459  1.38932533 -1.07228156]]]


In [68]:
# Ma trận Q, K, V
Q = x @ W_Q
print(f"Ma trận Query ban đầu:\n{Q}")
print(f"---------------------------")
print(f"Chiều của Q (batch, seq_len, d_model):\n{Q.shape}")

Ma trận Query ban đầu:
[[[ -0.53710413  -0.44015092  -0.98804689  -0.45771538  -3.7499716
    -1.68943888   2.23707953   2.93362574]
  [  1.07613691   4.50065157  -1.60473761   3.97270105 -11.74065036
     2.57401224   4.53750988   7.88176897]
  [ -1.49515518  -2.9262235    0.33355236  -2.17251732  -1.30006489
    -4.14838882   6.24622252   3.6636014 ]]

 [[ -1.88985252   0.70743996   1.23063365  -1.90659692   2.72218856
    -4.06102874  -0.88558092   1.01366204]
  [ -1.02930346  -1.95655445  -0.76167573  -2.56694174  -3.81260838
    -3.52118503   1.7186007    1.673128  ]
  [  0.80894037  -0.73624972   0.5353908    2.69640032  -1.19762344
     1.39722812   3.77534962   3.67034801]]]
---------------------------
Chiều của Q (batch, seq_len, d_model):
(2, 3, 8)


In [69]:
K = x @ W_K
print(f"Ma trận Key ban đầu:\n{K}")
print(f"--------------------------")
print(f"Chiều của K (batch, seq_len, num_groups * d_k):\n{K.shape}")


Ma trận Key ban đầu:
[[[  0.64682365  -0.2026224   -1.00948304  -2.68606074]
  [ -0.1807266   -0.05627826   2.0051453   -7.94092059]
  [ -1.32010081  -1.79878869  -9.24059834  -3.00829126]]

 [[ -3.89668036   5.58705609 -12.21677      6.84539295]
  [  0.60328553  -2.11988693   0.36429373  -3.42756367]
  [ -0.19946673  -1.04639334  -4.23584313  -2.78628345]]]
--------------------------
Chiều của K (batch, seq_len, num_groups * d_k):
(2, 3, 4)


In [70]:
V = x @ W_V
print(f"Ma trận Value ban đầu:\n{V}")
print(f"-----------------------")
print(f"Chiều của V (batch, seq_len, num_groups * d_k):\n{V.shape}")

Ma trận Value ban đầu:
[[[-1.09137438  2.84674419 -1.50248026  0.63681109]
  [ 2.85843161 -1.01830528 -5.40661755  1.11201523]
  [-2.966936    2.34474049  0.93127504  3.09409543]]

 [[-3.29497606 -0.52110381 -2.23467439 -1.62271873]
  [-0.34249387  2.63144559  2.07418722  1.4532188 ]
  [ 0.72445245 -1.98977546  0.24054463  3.90579942]]]
-----------------------
Chiều của V (batch, seq_len, num_groups * d_k):
(2, 3, 4)


In [71]:
# Có batch câu, mỗi câu có 5 từ, mỗi từ có h head
# mỗi head có d_k chiều
Q = Q.reshape(batch, seq_len, h, d_k)
print(f"Ma trận Q:\n{Q.shape}")
print(f"Có {batch} câu, mỗi câu có {seq_len} từ, mỗi từ có {h} head,\nmỗi head có {d_k} chiều")
for i in range(len(Q)):
    print(f"Câu {i}:\n{Q[i]}")

Ma trận Q:
(2, 3, 4, 2)
Có 2 câu, mỗi câu có 3 từ, mỗi từ có 4 head,
mỗi head có 2 chiều
Câu 0:
[[[ -0.53710413  -0.44015092]
  [ -0.98804689  -0.45771538]
  [ -3.7499716   -1.68943888]
  [  2.23707953   2.93362574]]

 [[  1.07613691   4.50065157]
  [ -1.60473761   3.97270105]
  [-11.74065036   2.57401224]
  [  4.53750988   7.88176897]]

 [[ -1.49515518  -2.9262235 ]
  [  0.33355236  -2.17251732]
  [ -1.30006489  -4.14838882]
  [  6.24622252   3.6636014 ]]]
Câu 1:
[[[-1.88985252  0.70743996]
  [ 1.23063365 -1.90659692]
  [ 2.72218856 -4.06102874]
  [-0.88558092  1.01366204]]

 [[-1.02930346 -1.95655445]
  [-0.76167573 -2.56694174]
  [-3.81260838 -3.52118503]
  [ 1.7186007   1.673128  ]]

 [[ 0.80894037 -0.73624972]
  [ 0.5353908   2.69640032]
  [-1.19762344  1.39722812]
  [ 3.77534962  3.67034801]]]


In [72]:
K = K.reshape(batch, seq_len, g, d_k)
print(f"Ma trận K:\n{K.shape}")
print(f"Có {batch} câu, mỗi câu có {seq_len} từ, mỗi từ có {g} group,\nmỗi group có {d_k} chiều")
for i in range(len(K)):
    print(f"Câu {i}:\n{K[i]}")

Ma trận K:
(2, 3, 2, 2)
Có 2 câu, mỗi câu có 3 từ, mỗi từ có 2 group,
mỗi group có 2 chiều
Câu 0:
[[[ 0.64682365 -0.2026224 ]
  [-1.00948304 -2.68606074]]

 [[-0.1807266  -0.05627826]
  [ 2.0051453  -7.94092059]]

 [[-1.32010081 -1.79878869]
  [-9.24059834 -3.00829126]]]
Câu 1:
[[[ -3.89668036   5.58705609]
  [-12.21677      6.84539295]]

 [[  0.60328553  -2.11988693]
  [  0.36429373  -3.42756367]]

 [[ -0.19946673  -1.04639334]
  [ -4.23584313  -2.78628345]]]


In [73]:
V = V.reshape(batch, seq_len, g, d_k)
print(f"Ma trận V:\n{V.shape}")
print(f"Có {batch} câu, mỗi câu có {seq_len} từ, mỗi từ có {g} group,\nmỗi group có {d_k} chiều")
for i in range(len(V)):
    print(f"Câu {i}:\n{V[i]}")

Ma trận V:
(2, 3, 2, 2)
Có 2 câu, mỗi câu có 3 từ, mỗi từ có 2 group,
mỗi group có 2 chiều
Câu 0:
[[[-1.09137438  2.84674419]
  [-1.50248026  0.63681109]]

 [[ 2.85843161 -1.01830528]
  [-5.40661755  1.11201523]]

 [[-2.966936    2.34474049]
  [ 0.93127504  3.09409543]]]
Câu 1:
[[[-3.29497606 -0.52110381]
  [-2.23467439 -1.62271873]]

 [[-0.34249387  2.63144559]
  [ 2.07418722  1.4532188 ]]

 [[ 0.72445245 -1.98977546]
  [ 0.24054463  3.90579942]]]


In [74]:
Q = Q.transpose(0, 2, 1, 3)
print(f"Ma trận Q:\n{Q.shape}")
print(f"Có {batch} mẫu, mỗi mẫu có {h} head, mỗi head có {seq_len} hàng\nđại diện cho 3 từ của câu,\nmỗi hàng có {d_k} chiều")
for i in range(len(Q)):
    print(f"Câu {i}:\n{Q[i]}")

Ma trận Q:
(2, 4, 3, 2)
Có 2 mẫu, mỗi mẫu có 4 head, mỗi head có 3 hàng
đại diện cho 3 từ của câu,
mỗi hàng có 2 chiều
Câu 0:
[[[ -0.53710413  -0.44015092]
  [  1.07613691   4.50065157]
  [ -1.49515518  -2.9262235 ]]

 [[ -0.98804689  -0.45771538]
  [ -1.60473761   3.97270105]
  [  0.33355236  -2.17251732]]

 [[ -3.7499716   -1.68943888]
  [-11.74065036   2.57401224]
  [ -1.30006489  -4.14838882]]

 [[  2.23707953   2.93362574]
  [  4.53750988   7.88176897]
  [  6.24622252   3.6636014 ]]]
Câu 1:
[[[-1.88985252  0.70743996]
  [-1.02930346 -1.95655445]
  [ 0.80894037 -0.73624972]]

 [[ 1.23063365 -1.90659692]
  [-0.76167573 -2.56694174]
  [ 0.5353908   2.69640032]]

 [[ 2.72218856 -4.06102874]
  [-3.81260838 -3.52118503]
  [-1.19762344  1.39722812]]

 [[-0.88558092  1.01366204]
  [ 1.7186007   1.673128  ]
  [ 3.77534962  3.67034801]]]


In [75]:
K = K.transpose(0, 2, 1, 3)
print(f"Ma trận K:\n{K.shape}")
print(f"Có {batch} mẫu, mỗi mẫu có {g} group, mỗi group có {seq_len} hàng\nđại diện cho 3 từ của câu,\nmỗi hàng có {d_k} chiều")
for i in range(len(K)):
    print(f"Câu {i}:\n{K[i]}")

Ma trận K:
(2, 2, 3, 2)
Có 2 mẫu, mỗi mẫu có 2 group, mỗi group có 3 hàng
đại diện cho 3 từ của câu,
mỗi hàng có 2 chiều
Câu 0:
[[[ 0.64682365 -0.2026224 ]
  [-0.1807266  -0.05627826]
  [-1.32010081 -1.79878869]]

 [[-1.00948304 -2.68606074]
  [ 2.0051453  -7.94092059]
  [-9.24059834 -3.00829126]]]
Câu 1:
[[[ -3.89668036   5.58705609]
  [  0.60328553  -2.11988693]
  [ -0.19946673  -1.04639334]]

 [[-12.21677      6.84539295]
  [  0.36429373  -3.42756367]
  [ -4.23584313  -2.78628345]]]


In [76]:
V = V.transpose(0, 2, 1, 3)
print(f"Ma trận V:\n{V.shape}")
print(f"Có {batch} mẫu, mỗi mẫu có {g} group, mỗi group có {seq_len} hàng\nđại diện cho 3 từ của câu,\nmỗi hàng có {d_k} chiều")
for i in range(len(V)):
    print(f"Câu {i}:\n{V[i]}")

Ma trận V:
(2, 2, 3, 2)
Có 2 mẫu, mỗi mẫu có 2 group, mỗi group có 3 hàng
đại diện cho 3 từ của câu,
mỗi hàng có 2 chiều
Câu 0:
[[[-1.09137438  2.84674419]
  [ 2.85843161 -1.01830528]
  [-2.966936    2.34474049]]

 [[-1.50248026  0.63681109]
  [-5.40661755  1.11201523]
  [ 0.93127504  3.09409543]]]
Câu 1:
[[[-3.29497606 -0.52110381]
  [-0.34249387  2.63144559]
  [ 0.72445245 -1.98977546]]

 [[-2.23467439 -1.62271873]
  [ 2.07418722  1.4532188 ]
  [ 0.24054463  3.90579942]]]


In [82]:
# Với head thứ nhất i = 0:
i = 0
group = i//(h//g)
print(f"Head {i} thuộc group: {group}")
Qi = Q[:, i, :, :]
print(f"Ma trận Q của head {i} cho mẫu 1:\n{Qi[0]}\nMa trận Q của head {i} cho mẫu 2:\n{Qi[1]}")
Ki = K[:, group, :, :]
print(f"Ma trận K của head {i} cho mẫu 1:\n{Ki[0]}\nMa trận K của head {i} cho mẫu 2:\n{Ki[1]}")
Vi = V[:, group, :, :]
print(f"Ma trận V của head {i} cho mẫu 1:\n{Vi[0]}\nMa trận V của head {i} cho mẫu 2:\n{Vi[1]}")

Head 0 thuộc group: 0
Ma trận Q của head 0 cho mẫu 1:
[[-0.53710413 -0.44015092]
 [ 1.07613691  4.50065157]
 [-1.49515518 -2.9262235 ]]
Ma trận Q của head 0 cho mẫu 2:
[[-1.88985252  0.70743996]
 [-1.02930346 -1.95655445]
 [ 0.80894037 -0.73624972]]
Ma trận K của head 0 cho mẫu 1:
[[ 0.64682365 -0.2026224 ]
 [-0.1807266  -0.05627826]
 [-1.32010081 -1.79878869]]
Ma trận K của head 0 cho mẫu 2:
[[-3.89668036  5.58705609]
 [ 0.60328553 -2.11988693]
 [-0.19946673 -1.04639334]]
Ma trận V của head 0 cho mẫu 1:
[[-1.09137438  2.84674419]
 [ 2.85843161 -1.01830528]
 [-2.966936    2.34474049]]
Ma trận V của head 0 cho mẫu 2:
[[-3.29497606 -0.52110381]
 [-0.34249387  2.63144559]
 [ 0.72445245 -1.98977546]]


In [83]:
# Với head thứ nhất i = 1:
i = 1
group = i//(h//g)
print(f"Head {i} thuộc group: {group}")
Qi = Q[:, i, :, :]
print(f"Ma trận Q của head {i} cho mẫu 1:\n{Qi[0]}\nMa trận Q của head {i} cho mẫu 2:\n{Qi[1]}")
Ki = K[:, group, :, :]
print(f"Ma trận K của head {i} cho mẫu 1:\n{Ki[0]}\nMa trận K của head {i} cho mẫu 2:\n{Ki[1]}")
Vi = V[:, group, :, :]
print(f"Ma trận V của head {i} cho mẫu 1:\n{Vi[0]}\nMa trận V của head {i} cho mẫu 2:\n{Vi[1]}")

Head 1 thuộc group: 0
Ma trận Q của head 1 cho mẫu 1:
[[-0.98804689 -0.45771538]
 [-1.60473761  3.97270105]
 [ 0.33355236 -2.17251732]]
Ma trận Q của head 1 cho mẫu 2:
[[ 1.23063365 -1.90659692]
 [-0.76167573 -2.56694174]
 [ 0.5353908   2.69640032]]
Ma trận K của head 1 cho mẫu 1:
[[ 0.64682365 -0.2026224 ]
 [-0.1807266  -0.05627826]
 [-1.32010081 -1.79878869]]
Ma trận K của head 1 cho mẫu 2:
[[-3.89668036  5.58705609]
 [ 0.60328553 -2.11988693]
 [-0.19946673 -1.04639334]]
Ma trận V của head 1 cho mẫu 1:
[[-1.09137438  2.84674419]
 [ 2.85843161 -1.01830528]
 [-2.966936    2.34474049]]
Ma trận V của head 1 cho mẫu 2:
[[-3.29497606 -0.52110381]
 [-0.34249387  2.63144559]
 [ 0.72445245 -1.98977546]]


==> Head 0 và Head 1 có cùng ma trận Key và Value, khác ma trận Query

In [ ]:
# Bước cuối ta tính scores và trọng số weights như trong MHA